In [54]:
import pandas as pd
from IPython.display import display

# CSV 파일 불러오기
value_file_path = r"C:\datasets\kor_value.csv"
ticker_file_path = r"C:\datasets\kor_ticker.csv"

data = pd.read_csv(value_file_path)
ticker_data = pd.read_csv(ticker_file_path)

# 종목코드를 문자열로 변환하여 데이터 타입을 맞춤
data['종목코드'] = data['종목코드'].astype(str)
ticker_data['종목코드'] = ticker_data['종목코드'].astype(str)

# value의 종목코드를 ticker와 같은 6자리 형식으로 변경 (.unique로 확인)
data['종목코드'] = data['종목코드'].str.zfill(6)

#조건 필터링
dy_filtered = data[(data['지표'] == 'DY') & (data['값'] >= 0.04)]
pbr_filtered = data[(data['지표'] == 'PBR') & (data['값'] > 0)  & (data['값'] <= 1)]
per_filtered = data[(data['지표'] == 'PER') & (data['값'] > 0)  & (data['값'] <= 15)]

# 조건을 모두 만족하는 종목 코드 찾기
growth_stock_codes = set(dy_filtered['종목코드']) & set(pbr_filtered['종목코드']) & set(per_filtered['종목코드'])

# 조건을 만족하는 종목만 선택하고, 필요한 지표만 포함
filtered_data = data[(data['종목코드'].isin(growth_stock_codes)) & (data['지표'].isin(['DY', 'PBR', 'PER']))]

# 테이블 재구성: 종목코드, 기준일을 인덱스로 두고 지표를 열로 변환
pivoted_data = filtered_data.pivot_table(index=['종목코드', '기준일'], columns='지표', values='값', fill_value=0).reset_index()

# 종목코드 기준으로 종목명을 병합
pivoted_data = pivoted_data.merge(ticker_data[['종목코드', '종목명']], on='종목코드', how='left')

# 원하는 열 순서로 재정렬
pivoted_data = pivoted_data[['종목코드', '종목명', '기준일', 'PBR', 'PER', 'DY']]

#DY를 내림차순으로 정렬
pivoted_data = pivoted_data.sort_values(by='DY', ascending=False).reset_index(drop=True)

# 결과 테이블로 출력
display(pivoted_data)

# CSV 파일로 저장
output_file_path = r"C:\datasets\filtered_dividend_stocks.csv"
pivoted_data.to_csv(output_file_path, index=False, encoding='utf-8-sig')

,종목코드,종목명,기준일,PBR,PER,DY
0,007330,푸른저축은행,2024-07-03,0.4270,12.0799,0.0738
1,001230,동국홀딩스,2024-07-03,0.1662,0.4557,0.0719
2,037710,광주신세계,2024-07-03,0.2985,5.3751,0.0719
3,035000,HS애드,2024-07-03,0.5915,6.4034,0.0714
4,003540,대신증권,2024-07-03,0.2907,6.2770,0.0710
...,...,...,...,...,...,...
92,035810,이지홀딩스,2024-07-03,0.1852,2.8689,0.0406
93,039240,경남스틸,2024-07-03,0.7500,13.2023,0.0402
94,023000,삼원강재,2024-07-03,0.4503,6.1808,0.0402
95,029530,신도리코,2024-07-03,0.3762,6.7561,0.0401


In [55]:
import pandas as pd

# CSV 파일 읽기
csv_file_path = r"C:\datasets\filtered_dividend_stocks.csv"
data = pd.read_csv(csv_file_path, delimiter="\t")  # 탭 구분자로 읽어오기

# JSON 파일로 저장하기
json_file_path = r"C:\datasets\filtered_dividend_stocks.json"
data.to_json(json_file_path, orient="records", force_ascii=False, indent=4)

print(f"JSON 파일이 '{json_file_path}' 이름으로 저장되었습니다.")

JSON 파일이 'C:\datasets\filtered_dividend_stocks.json' 이름으로 저장되었습니다.


In [56]:
import json
import pandas as pd

# JSON 파일 경로
input_file = r"C:\datasets\filtered_dividend_stocks.json"
output_file = r"C:\datasets\filtered_dividend_stocks.json"

# JSON 파일 읽기
with open(input_file, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# 문자열로 결합된 데이터를 분리하여 각 열로 변환
processed_data = []
for item in raw_data:
    values = list(item.values())[0].split(',')
    processed_data.append({
        "종목코드": int(values[0]),
        "종목명": values[1],
        "기준일": values[2],
        "PBR": float(values[3]),
        "PER": float(values[4]),
        "DY": float(values[5])
    })

# DataFrame으로 변환 후 JSON으로 저장
df = pd.DataFrame(processed_data)
df.to_json(output_file, orient="records", force_ascii=False, indent=4)

print(f"필드별로 분리된 JSON 파일이 '{output_file}'에 저장되었습니다.")

필드별로 분리된 JSON 파일이 'C:\datasets\filtered_dividend_stocks.json'에 저장되었습니다.
